# Fall 2024 Data Science Track: Week 5 - Unsupervised Learning

## Packages, Packages, Packages!

Import *all* the things here! You need: `matplotlib`, `networkx`, `numpy`, and `pandas`―and also `ast.literal_eval` to correctly deserialize two columns in the `rules.tsv.xz` file.

If you got more stuff you want to use, add them here too. 🙂

In [1]:
# Don’t worry about this. This is needed to interpret the Python code that is embedded in the data set. You only need it literally in the very next code cell and nowhere else. 
from ast import literal_eval

# The rest is just the stuff from the lecture.
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

# Instacart Association Rules

## Introduction

With the packages out of the way, now you will be working with the Instacart association rules data set, mined from the [Instacart Market Basket Analysis data set](https://www.kaggle.com/c/instacart-market-basket-analysis/data) on Kaggle. [The script](https://github.com/LiKenun/shopping-assistant/blob/main/api/preprocess_instacart_market_basket_analysis_data.py) that does it and the instructions to run it can be found in my [Shopping Assistant Project](https://github.com/LiKenun/shopping-assistant) repository.

## Load the Data

This code has already been pre-written, simply because there are a few quirks which require converters to ensure the correct deserialization of some columns.

In [2]:
rules_data_path = 'data/rules.tsv.xz'                       # You do not need to decompress this yourself. Pandas understands how to read compressed data.

df_rules = pd.read_csv(rules_data_path,
                       sep='\t',
                       quoting=3,                           # This disables interpretation of quotes by Pandas itself, because both single and double quotes will be resolved by literal_eval.
                       converters={
                           'consequent_item': literal_eval,
                           'antecedent_items': literal_eval # This reads something like ["Grandma's 8\" Chip Cookies", '6" Apple Pie'] into a list, so you will get a column where each individual cell is a list.
                       },
                       low_memory=True)                     # For Chris

But just *how* many rules were just loaded‽

In [3]:
# Show the list of column names and the number of rules.

print(df_rules.columns)

len(df_rules)

Index(['consequent_item', 'transaction_count', 'item_set_count',
       'antecedent_count', 'consequent_count', 'antecedent_items'],
      dtype='object')


1048575

## Metrics

Compute the support, confidence, and lift of each rule.

* The rule’s *support* tells you how frequently the set of items appears in the dataset. It’s important to prune infrequent sets from further consideration.
    * The simple definition: $$P(A \cap B)$$
    * `= item_set_count / transaction_count`
* The rule’s *confidence* tells you how often a the rule is true. Divide the support for the set of items by the support for just the antecedents. Rules which are not true very often are also pruned.
    * The simple definition: $$\frac{P(A \cap B)}{P(A)}$$
    * `= item_set_count / transaction_count / (antecedent_count / transaction_count)`
    * `= item_set_count / antecedent_count`
* The rule’s *lift* tells you how much more likely the consequent is, given the antecedents, compared to its baseline probability. Divide the support for the set of items by both the support of the antecedents and consequent. Equivalently, divide the confidence by the support of the consequent.
    * The simple definition: $$\frac{P(A \cap B)}{P(A) \cdot P(B)}$$
    * `= item_set_count / transaction_count / (antecedent_count / transaction_count * (consequent_count / transaction_count))`
    * `= item_set_count / antecedent_count / (consequent_count / transaction_count)`
    * `= item_set_count * transaction_count / (antecedent_count * consequent_count)`

In [4]:
# Add new columns support, confidence, and lift to df_rules. And show the first 50 rules.

df_rules["support"] = df_rules["item_set_count"] / df_rules["transaction_count"]
df_rules["confidence"] = df_rules["item_set_count"] / df_rules["antecedent_count"]
df_rules["lift"] = df_rules["item_set_count"] * df_rules["transaction_count"] / (df_rules["antecedent_count"] * df_rules["consequent_count"])
df_rules.head(50)

consequent_item  transaction_count  \
0   Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
1   Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
2    Total 0% with Honey Nonfat Greek Strained Yogurt            3346083   
3                           Total 0% Raspberry Yogurt            3346083   
4                                 Pineapple Yogurt 2%            3346083   
5                          Fat Free Strawberry Yogurt            3346083   
6                           Fat Free Blueberry Yogurt            3346083   
7   Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
8   Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
9   Total 2% Greek Strained Yogurt with Cherry 5.3 oz            3346083   
10  Total 2% All Natural Greek Strained Yogurt wit...            3346083   
11                          Total 0% Raspberry Yogurt            3346083   
12                                Pineapple Yogurt 2%            3346083   
13                         Fat Free Strawberry Yogurt            3346083   
14                                  Blackberry Yogurt            3346083   
15  Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
16  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
17   Total 0% with Honey Nonfat Greek Strained Yogurt            3346083   
18                          Total 0% Raspberry Yogurt            3346083   
19                                Pineapple Yogurt 2%            3346083   
20                         Fat Free Strawberry Yogurt            3346083   
21                          Fat Free Blueberry Yogurt            3346083   
22  Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
23  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
24  Total 2% Greek Strained Yogurt with Cherry 5.3 oz            3346083   
25  Total 2% All Natural Greek Strained Yogurt wit...            3346083   
26                          Total 0% Raspberry Yogurt            3346083   
27                                Pineapple Yogurt 2%            3346083   
28                         Fat Free Strawberry Yogurt            3346083   
29                                  Blackberry Yogurt            3346083   
30  Total 2% with Strawberry Lowfat Greek Strained...            3346083   
31  Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
32   Total 2% Lowfat Greek Strained Yogurt with Peach            3346083   
33  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
34  Total 2% Greek Strained Yogurt with Cherry 5.3 oz            3346083   
35  Total 2% All Natural Greek Strained Yogurt wit...            3346083   
36                          Total 0% Raspberry Yogurt            3346083   
37                         Fat Free Strawberry Yogurt            3346083   
38                                  Blackberry Yogurt            3346083   
39  Total 2% with Strawberry Lowfat Greek Strained...            3346083   
40  Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
41   Total 2% Lowfat Greek Strained Yogurt with Peach            3346083   
42  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
43  Total 2% Greek Strained Yogurt with Cherry 5.3 oz            3346083   
44  Total 2% All Natural Greek Strained Yogurt wit...            3346083   
45                          Total 0% Raspberry Yogurt            3346083   
46                         Fat Free Strawberry Yogurt            3346083   
47                                  Blackberry Yogurt            3346083   
48  Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
49  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   

    item_set_count  antecedent_count  consequent_count  \
0              101               123               128   
1              101               128         

The yogurts have got some insane lift (*over 9,000*). Why do you think that might be?

*(Write your answer here.)*

Yogurts have insane lift because people don't just buy one container of yogurt. People will buy a variety of flavors of yogurts and this has to do with the fact that yogurts are a popular, on-the-go, easy breakfast item. When purchasing, the customer is probably planning for the week or month ahead and will buy yogurts of varying flavors (yk just so they don't eat the same flavor for the whole month). Other things we can think about are promos, sales, deals like BOGO, etc. Perhaps if a customer buys pineapple yogurt and raspberry yogurt, they'll get 50% off. Lots to think about, but I am on board w/ the yogurt hype fr.

In [5]:
# Query the rule set if you have to to find out more.

high_lifts = df_rules[df_rules["lift"] > 9000].head(5)
print(high_lifts)

high_confidence = df_rules[df_rules["confidence"] > 0.8].head(5)
print(high_confidence)



                                     consequent_item  transaction_count  \
0  Total 2% with Raspberry Pomegranate Lowfat Gre...            3346083   
1  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
2   Total 0% with Honey Nonfat Greek Strained Yogurt            3346083   
3                          Total 0% Raspberry Yogurt            3346083   
4                                Pineapple Yogurt 2%            3346083   

   item_set_count  antecedent_count  consequent_count  \
0             101               123               128   
1             101               128               123   
2             101               123               128   
3             101               123               128   
4             101               128               123   

                                    antecedent_items  support  confidence  \
0  [Fat Free Blueberry Yogurt, Pineapple Yogurt 2...  0.00003    0.821138   
1  [Fat Free Strawberry Yogurt, Total 0% Raspberr...

## Network Visualization for Consequents with Single Antecedents

Let’s now visualize a small subset of 1,000,000+ rules. First, filter the rule set for the following to whittle it down to something more manageable:

1. The rule must have exactly `1` antecedent item. (There should be 38,684 such rules.)
2. The lift must be between `5` and `20`. (There should be 1,596 such rules, including the prior criterion.)
3. Either the antecedent or consequent of the rule must contain `'Hummus'`, but not both. (This should get you down to 26 rules.)
    * Convert the antecedents `list`-typed column to a `str`-typed column (`antecedent_item`) since there will only be a single antecedent in the subset.
    * Replace any item containing `'Hummus'` to just `'Hummus'`. This will make the visualization more readable later.

Hint: your code may run more efficiently if you re-order certain processing steps.

Assign the subset to `df_rules_subset`.

In [23]:
# Define df_rules_subset.

# filter for rules with exactly 1 antecedent item & lift btwn 5 and 20
df_rules_subset = df_rules[
    (df_rules['antecedent_items'].apply(lambda x: len(x) == 1)) &
    (df_rules['lift'] >= 5) & 
    (df_rules['lift'] <= 20)
]


df_rules_subset["antecedent_items"] = df_rules_subset["antecedent_items"].apply(lambda x: str(x[0]))
df_rules_subset["antecedent_items"] = df_rules_subset["antecedent_items"].apply(lambda x: 'Hummus' if 'Hummus' in x else x)
df_rules_subset["consequent_item"] = df_rules_subset["consequent_item"].apply(lambda x: 'Hummus' if 'Hummus' in x else x)

df_rules_subset = df_rules_subset[(df_rules_subset['antecedent_items'].apply(lambda x: 'Hummus' in x) ^ df_rules_subset["consequent_item"].apply(lambda x: 'Hummus' in x))]

df_rules_subset


C:\Users\zaraa\AppData\Local\Temp\ipykernel_38116\2947064671.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rules_subset["antecedent_items"] = df_rules_subset["antecedent_items"].apply(lambda x: str(x[0]))
C:\Users\zaraa\AppData\Local\Temp\ipykernel_38116\2947064671.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rules_subset["antecedent_items"] = df_rules_subset["antecedent_items"].apply(lambda x: 'Hummus' if 'Hummus' in x else x)
C:\Users\zaraa\AppData\Local\Temp\ipykernel_38116\294706467

consequent_item  transaction_count  \
377756  Total 2% Lowfat Greek Strained Yogurt With Blu...            3346083   
387837                                        Clementines            3346083   
416420                                             Hummus            3346083   
419119                                             Hummus            3346083   
422178                                             Hummus            3346083   
440198                                             Hummus            3346083   
442099                               Organic Baby Carrots            3346083   
463035                                             Hummus            3346083   
463988                                             Hummus            3346083   
476420                                             Hummus            3346083   
506520                                             Hummus            3346083   
519416                                             Hummus            3346083   
523791                                             Hummus            3346083   
531448                                             Hummus            3346083   
546619                                             Hummus            3346083   
547477                                             Hummus            3346083   
550256                               Organic Baby Carrots            3346083   
554787                                             Hummus            3346083   
567246                                             Hummus            3346083   
570788                         Organic Small Bunch Celery            3346083   
570961                                             Hummus            3346083   
571790                                             Hummus            3346083   
572363                                             Hummus            3346083   
573155                                             Hummus            3346083   
587069                               Organic Baby Carrots            3346083   

        item_set_count  antecedent_count  consequent_count  \
377756             100               887             21405   
387837             174              1334             32194   
416420             345              1656             74172   
419119             293              1434             74172   
422178             331              1655             74172   
440198            3119             17333             74172   
442099             258              1334             80493   
463035             151               933             74172   
463988             311              1929             74172   
476420             512              3333             74172   
506520             168              1208             74172   
519416             117               875             74172   
523791            1544             11702             74172   
531448             166              1287             74172   
546619             244              1973             74172   
547477             104               843             74172   
550256             122               918             80493   
554787             250              2066             74172   
567246             221              1887             74172   
570788             109               985             70714   
570961             126              1086             74172   
571790             287              2479             74172   
572363             257              2223             74172   
573155             111               962             74172   
587069             235              1945             80493   

                                         antecedent_items   support  \
377756                                             Hummus  0.000030   
387837                                             Hummus  0.000052   
416420                           Organic White Pita Bread  0.000103   
419119                           Organic Whole Wheat Pita  0.000

Build a network `graph_rules_subset` from the association rules subset.

In [28]:
# Define graph_rules_subset, add the graph’s edges, and plot it. You may need a large figure size, smaller node size, and smaller font size.

graph_rules_subset = nx.MultiDiGraph()
edges = [
    (row['antecedent_items'], row['consequent_item']) for _, row in df_rules_subset.iterrows()
]
graph_rules_subset.add_edges_from(edges)

# Then render the graph.
plt.figure(figsize=(12, 12))  # Set figure size for better readability

# Position nodes using the spring layout (force-directed)
pos = nx.spring_layout(graph_rules_subset, k=0.5, iterations=50)

# Draw the nodes
nx.draw_networkx_nodes(graph_rules_subset, pos, node_size=700, node_color='green')

# Draw the edges
edges = nx.draw_networkx_edges(graph_rules_subset, pos, arrowstyle='->', arrowsize=20, edge_color='gray', width=2)

# Draw node labels with smaller font size
nx.draw_networkx_labels(graph_rules_subset, pos, font_size=8)


# Display the graph
plt.title("Network Graph of Association Rules Subset", fontsize=14)
plt.axis("off")  # Hide the axes
plt.show()

What can you tell about people who buy hummus?

*(Write your answer here.)*

People who buy hummus tend to also buy pitas, crackers, chips, suggesting that they are likely going to pair their hummus dip with these snacks.

## Make a Prediction

Given that the basket of items contains the following items, use the full set of association rules to predict the next 20 most likely items (consequents) that the person will add to the basket in descending order of lift:

* `'Orange Bell Pepper'`
* `'Organic Red Bell Pepper'`

Hint: a single item in the basket may be a better predictor of some consequents than both items considered together. You must consider both or either, but not neither.

In [30]:
basket = {'Orange Bell Pepper', 'Organic Red Bell Pepper'}

c1 = df_rules['antecedent_items'].apply(lambda x: 'Organic Red Bell Pepper' in x)
c2 = df_rules['antecedent_items'].apply(lambda x: 'Orange Bell Pepper' in x)

df_rules[# A few conditions go here which takes care of all the following cases:
         # * Just Orange Bell Pepper
         # * Just Organic Red Bell Pepper
         # * Both Orange Bell Pepper and Organic Red Bell Pepper
         # You can do it using just 2 conditions. :)
         c1 | c2
         ] \
        .sort_values('lift', ascending=False) \
        .head(20)

consequent_item  transaction_count  \
342462      Icelandic Style Skyr Blueberry Non-fat Yogurt            3346083   
345882      Icelandic Style Skyr Blueberry Non-fat Yogurt            3346083   
346832           Nonfat Icelandic Style Strawberry Yogurt            3346083   
346901                           Non Fat Raspberry Yogurt            3346083   
349532      Icelandic Style Skyr Blueberry Non-fat Yogurt            3346083   
350852      Icelandic Style Skyr Blueberry Non-fat Yogurt            3346083   
350991                           Non Fat Raspberry Yogurt            3346083   
351360      Icelandic Style Skyr Blueberry Non-fat Yogurt            3346083   
351433           Nonfat Icelandic Style Strawberry Yogurt            3346083   
351881      Icelandic Style Skyr Blueberry Non-fat Yogurt            3346083   
352283                         Vanilla Skyr Nonfat Yogurt            3346083   
352884                        Organic Frozen Mango Chunks            3346083   
353323                         Vanilla Skyr Nonfat Yogurt            3346083   
353543                         Organic Orange Bell Pepper            3346083   
353580                           Non Fat Raspberry Yogurt            3346083   
353760                             Organic Sliced Peaches            3346083   
354421  Total 2% with Strawberry Lowfat Greek Strained...            3346083   
355427                                Organic Bell Pepper            3346083   
355819                                Organic Bell Pepper            3346083   
356261                                 Yellow Bell Pepper            3346083   

        item_set_count  antecedent_count  consequent_count  \
342462             100               209             19962   
345882             132               311             19962   
346832             100               441             11025   
346901             184               529             16953   
349532             184               494             19962   
350852             167               471             19962   
350991             132               441             16953   
351360             101               291             19962   
351433             101               529             11025   
351881             133               392             19962   
352283             167               529             18899   
352884             109               612             10938   
353323             133               441             18899   
353543             145               834             11005   
353580             126               471             16953   
353760             109               676             10300   
354421             235               502             30866   
355427             114               324             24331   
355819             145               420             24331   
356261             109               295             26625   

                                         antecedent_items   support  \
342462  [Nonfat Icelandic Style Strawberry Yogurt, Ora...  0.000030   
345882     [Non Fat Raspberry Yogurt, Orange Bell Pepper]  0.000039   
346832  [Icelandic Style Skyr Blueberry Non-fat Yogurt...  0.000030   
346901  [Icelandic Style Skyr Blueberry Non-fat Yogurt...  0.000055   
349532  [Non Fat Raspberry Yogurt, Organic Red Bell Pe...  0.000055   
350852  [Organic Red Bell Pepper, Vanilla Skyr Nonfat ...  0.000050   
350991  [Icelandic Style Skyr Blueberry Non-fat Yogurt...  0.000039   
351360  [Nonfat Icelandic Style Strawberry Yogurt, Org...  0.000030   
351433  [Icelandic Style Skyr Blueberry Non-fat Yogurt...  0.000030   
351881   [Orange Bell Pepper, Vanilla Skyr Nonfat Yogurt]  0.000040   
352283  [Icelandic Style Skyr Blueberry Non-fat Yogurt...  0.000050   
352884  [Organic Red Bell Pepper, Organic Sliced Peaches]  0.000033   
353323  [Icelandic Style Skyr Blueberry Non-fat Yogurt...  0.000040   
353543  [Organic Bell Pepper, Organic Raspberries, Org...  0.000

## Bonus: Other Interesting Findings

Find and share something else interesting about these association rules. It can be a graph, table, or some other format that illustrates your point.